In [3]:
#import packages
from libchebipy import ChebiEntity
import libchebipy
import pubchempy as pcp
import io
import requests
import json
import cobra
import functions
from functions import *

In [4]:
model=cobra.io.load_matlab_model('Data/iFpraus_v_1_0.mat')
info(model)

Metabolites :  833
Reactions :  1030
Genes :  602
Compartments :  {'c': '', 'e': ''}
Objective function : 
 Maximize
1.0*Biomass_FP - 1.0*Biomass_FP_reverse_ee33b 



---
## mMCB

(There are also supplements mentioned in the paper but this is the base medium)
* [Bacteriological Peptome (6.5) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN): 
    * Vague, decided to go with polypeptides: <b>polypep</b>
    * Nitrogen: <b>n2</b>
* [Soy Peptome (5.0) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0044&cat=&c=UK&lang=EN) --> another vague one:
    * Stachyose: <b>stys</b>
    * Raffinose: <b>raffin</b>
    * Sucrose: <b>sucr</b>
    * Nitrogen: <b>n2</b>
* [Yeast Extract (3.0) (VWR International, Darmstadt, Germany)](https://us.vwr.com/store/product/7437401/vwr-life-science-yeast-extract-bacteriological-grade):
    * Vitamin B (same logic as RCM):
        * 1. Thiamin -> <b>thm</b>
        * 2. Riboflavin -> <b>ribflv</b>
        * 3. Niacin -> <b>trp__L</b> (made of tryptophan)
        * 5. Pantothenic Acid -> <b>pnto__R</b>
        * 6. Pyridoxine -> <b>pydxn</b>
        * 7. Biotin -> <b>btn</b>
        * 9. Folic Acid -> ... can't find anything
        * 12. Cobalamin -> <b>b12</b> (or cbl1)
* [Tryptone (2.5) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0042&c=UK&lang=EN): 
    * Tryptophan: <b>trp__L</b>
* NaCL (1.5) (VWR International, Darmstadt, Germany):
    * NaCl: <b>na1, cl</b>
* K<sub>2</sub>HPO<sub>4</sub> (1.0) (Merck International, Darmstadt, Germany):
    * Potassium: <b>k</b>
    * Phosphate: <b>p1</b>
    * Hydrogen: <b>h2</b>
* KH<sub>2</sub>PO<sub>4</sub> (1.0) (Merck International, Darmstadt, Germany):
    * Potassium: <b>k</b>
    * Phosphate: <b>p1</b>
    * Hydrogen: <b>h2</b>
* Na<sub>2</sub>SO<sub>4</sub> (2.0) (VWR):
    * Sodium: <b>na1</b>
    * Sulfate: <b>so4</b>
* MgSO<sub>4</sub>*7H<sub>2</sub>O (1.0) (Merck):
    * Magnesium: <b>mg2</b>
    * Sulfate: <b>so4</b>
* CaCl<sub>2</sub>*2H<sub>2</sub>O (0.1) (Merck):
    * Calcium Chloride: <b>ca2, cl</b>
* NH<sub>4</sub>Cl (1.0) (Merck):
    * Ammonium Chloride: <b>nh4, cl</b>
* Cysteine-HCL (0.4) (Merck):
    * Cysteine: <b>cys__L</b>
    * HCL: <b>h2, cl</b>
* NaHCO<sub>3</sub> (0.2) (VWR):
    * Sodium: <b>na1</b>
    * Bicarbonate: <b>hco3<b>
* MnSO<sub>4</sub>*H<sub>2</sub>O (0.05) (VWR):
    * Manganese: <b>mn2</b>
    * Sulfate: <b>so4</b>
* FeSO<sub>4</sub>*7H<sub>2</sub>O (0.005) (Merck):
    * Iron: <b>fe</b>
    * Sulfate: <b>so4</b>
* ZnSO<sub>4</sub>*7H<sub>2</sub>O (0.005) (VWR):
    * Zinc: <b>zn2</b>
    * Sulfate: <b>so4</b>
* Hemin (0.005) (Sigma-Aldrich, Steinheim, Germany):
    * ... contains iron
* Menadione (0.005) (S-A):
    * <b>mndn</b>
* Resazurin (0.001) (S-A):
    * Fluoro identifier



| Component                            | Concentration (g/L) |
|--------------------------------------|---------------------|
| Bacteriological Peptome              | 6.5                 |
| Soy Peptome                          | 5.0                 |
| Yeast Extract                        | 3.0                 |
| Tryptone                             | 2.5                 |
| NaCL                                 | 1.5                 |
| K<sub>2</sub>PO<sub>4</sub>          | 1.0                 |
| KH<sub>2</sub>PO<sub>4</sub>         | 1.0                 |
| Na<sub>2</sub>SO<sub>4</sub>         | 2.0                 |
| Mg<sub>2</sub>SO<sub>4</sub>         | 1.0                 |
| CaCl<sub>2</sub>*2H<sub>2</sub>O     | 0.1                 |
| NH<sub>4</sub>Cl                     | 1.0                 |
| Cysteine-HCL                         | 0.4                 |
| NaHCO<sub>3</sub>                    | 0.2                 |
| MnSO<sub>4</sub>*H<sub>2</sub>O      | 0.05                |
| FeSOMnSO<sub>4</sub>*7H<sub>2</sub>O | 0.005               |
| ZnSO<sub>4</sub>*7H<sub>2</sub>O     | 0.005               |
| Hemin                                | 0.005               |
| Menadione                            | 0.005               |
| Resazurin                            | 0.001               |


In [6]:
# Get medium from model reactions
root_model = model.copy()
current_medium = [metab.lstrip('EX_') for metab in list(root_model.medium.keys())]

# Check which metabolites control growth by removing one at a time and seeing the results
def growth_stoppers(model, metabolites):
    results = {}
    for metabolite in metabolites:
        metabolites.remove(metabolite.lstrip('EX_'))
        test_model = medium(model, metabolites)
        results[metabolite.lstrip('EX_')] = test_model.slim_optimize()
        metabolites = [metab.lstrip("EX_") for metab in metabolites]
        metabolites.append(metabolite.lstrip('EX_'))

    import pandas as pd
    results_df = pd.DataFrame(results.items(), columns=["metab","growth"]).sort_values('growth', ascending=False)

    return results_df

growth_check_df = growth_stoppers(root_model, current_medium)

# Filtering the dataframe to find the metabolites that hinder growth the most
# (If growth is less than the mean, then it's considered to be enough to consider not removing it from the theoretical medium)
# Don't take these out of the medium if you can help it:
mandatory = growth_check_df[growth_check_df['growth'] < growth_check_df['growth'].mean()]
mandatory

,metab,growth
64,glyc3p(e),5.518982e+00
119,pi(e),5.518982e+00
110,nac(e),7.879200e-13
141,trp_L(e),6.209833e-15
87,k(e),3.821436e-15
41,cobalt2(e),3.598136e-15
43,cu2(e),3.598136e-15
50,fol(e),1.340104e-15
131,ser_L(e),-2.866077e-15
39,cl(e),-2.943929e-15


In [7]:
### Remove metabolites from the medium that can't be justified
# This part is where the metabolites for each component of the media are removeed
# This is entirely subjective and should be based on some kind of prior knowledge
# (Please don't delete lines, simply comment them out <3)

# current_medium.remove('EX_3mop(e)')
current_medium.remove('EX_4abz(e)')
current_medium.remove('EX_5oxpro(e)')
# current_medium.remove('EX_Lcyst(e)')
# current_medium.remove('EX_Lcystin(e)')
# current_medium.remove('EX_ac(e)')
# current_medium.remove('EX_acasp(e)')
# current_medium.remove('EX_acgal(e)')
current_medium.remove('EX_acgalglcur(e)')
# current_medium.remove('EX_acgam(e)')
# current_medium.remove('EX_acglu(e)')
# current_medium.remove('EX_acnam(e)')
# current_medium.remove('EX_ade(e)')
current_medium.remove('EX_adocbl(e)')
# current_medium.remove('EX_ala_D(e)')
# current_medium.remove('EX_ala_L(e)')
# current_medium.remove('EX_alaala(e)')
# current_medium.remove('EX_alaasp(e)')
# current_medium.remove('EX_alagln(e)')
# current_medium.remove('EX_alaglu(e)')
# current_medium.remove('EX_alagly(e)')
# current_medium.remove('EX_alahis(e)')
# current_medium.remove('EX_alaleu(e)')
# current_medium.remove('EX_alathr(e)')
current_medium.remove('EX_arab_L(e)')
# current_medium.remove('EX_arg_L(e)')
# current_medium.remove('EX_asn_L(e)')
current_medium.remove('EX_aso3(e)')
current_medium.remove('EX_aso4(e)')
# current_medium.remove('EX_asp_L(e)')
# current_medium.remove('EX_btn(e)')
current_medium.remove('EX_but(e)')
current_medium.remove('EX_butso3(e)')
# current_medium.remove('EX_ca2(e)')
current_medium.remove('EX_cbl1(e)')
current_medium.remove('EX_cbl2(e)')
current_medium.remove('EX_cd2(e)')
current_medium.remove('EX_cellb(e)')
# current_medium.remove('EX_cgly(e)')
current_medium.remove('EX_cit(e)')
# current_medium.remove('EX_cl(e)')
current_medium.remove('EX_co2(e)')
# current_medium.remove('EX_cobalt2(e)') ###
# current_medium.remove('EX_cps_fp(e)') ###
# current_medium.remove('EX_cu2(e)')
# current_medium.remove('EX_cys_L(e)')
current_medium.remove('EX_dhor_S(e)')
current_medium.remove('EX_ethso3(e)')
current_medium.remove('EX_fe2(e)')
# current_medium.remove('EX_fe3(e)')
current_medium.remove('EX_fe3dcit(e)')
# current_medium.remove('EX_fol(e)')
# current_medium.remove('EX_for(e)')
current_medium.remove('EX_fru(e)')
current_medium.remove('EX_fum(e)')
current_medium.remove('EX_gal(e)')
current_medium.remove('EX_galur(e)')
# current_medium.remove('EX_gam(e)')
# current_medium.remove('EX_glc(e)')
# current_medium.remove('EX_glcur(e)')
# current_medium.remove('EX_gln_L(e)')
# current_medium.remove('EX_glu_L(e)')
# current_medium.remove('EX_gly(e)')
# current_medium.remove('EX_glyasn(e)')
# current_medium.remove('EX_glyasp(e)')
# current_medium.remove('EX_glyc3p(e)') ###
current_medium.remove('EX_glyc_R(e)')
# current_medium.remove('EX_glygln(e)')
# current_medium.remove('EX_glyglu(e)')
# current_medium.remove('EX_glygly(e)')
# current_medium.remove('EX_glyleu(e)')
# current_medium.remove('EX_glymet(e)')
# current_medium.remove('EX_glyphe(e)')
# current_medium.remove('EX_glypro(e)')
# current_medium.remove('EX_glytyr(e)')
current_medium.remove('EX_gthox(e)')
current_medium.remove('EX_gthrd(e)')
# current_medium.remove('EX_gua(e)')
# current_medium.remove('EX_h(e)')
# current_medium.remove('EX_h2o(e)')
current_medium.remove('EX_h2s(e)')
current_medium.remove('EX_hg2(e)')
current_medium.remove('EX_his_L(e)')
current_medium.remove('EX_hxan(e)')
# current_medium.remove('EX_ile_L(e)')
current_medium.remove('EX_ins(e)')
current_medium.remove('EX_inulin(e)')
current_medium.remove('EX_isetac(e)')
# current_medium.remove('EX_k(e)')
current_medium.remove('EX_kesto(e)')
current_medium.remove('EX_kestopt(e)')
current_medium.remove('EX_kestottr(e)')
current_medium.remove('EX_lac_D(e)')
current_medium.remove('EX_lcts(e)')
# current_medium.remove('EX_leu_L(e)')
# current_medium.remove('EX_leugly(e)')
# current_medium.remove('EX_leuleu(e)')
# current_medium.remove('EX_lys_L(e)')
current_medium.remove('EX_mal_L(e)')
current_medium.remove('EX_malt(e)')
current_medium.remove('EX_man(e)')
current_medium.remove('EX_meoh(e)')
# current_medium.remove('EX_met_D(e)')
# current_medium.remove('EX_met_L(e)')
# current_medium.remove('EX_metala(e)')
# current_medium.remove('EX_metsox_R_L(e)')
# current_medium.remove('EX_metsox_S_L(e)')
# current_medium.remove('EX_mg2(e)')
current_medium.remove('EX_mobd(e)')
current_medium.remove('EX_mso3(e)')
# current_medium.remove('EX_na1(e)')
# current_medium.remove('EX_nac(e)') ###
current_medium.remove('EX_ncam(e)')
# current_medium.remove('EX_nh4(e)')
current_medium.remove('EX_o2(e)')
current_medium.remove('EX_orn(e)')
current_medium.remove('EX_orot(e)')
# current_medium.remove('EX_pb(e)')
# current_medium.remove('EX_pect(e)') ###
# current_medium.remove('EX_phe_L(e)')
# current_medium.remove('EX_pi(e)')
current_medium.remove('EX_plac(e)')
current_medium.remove('EX_pnto_R(e)')
# current_medium.remove('EX_pro_L(e)')
# current_medium.remove('EX_progly(e)')
current_medium.remove('EX_ptrc(e)')
# current_medium.remove('EX_pydam(e)')
# current_medium.remove('EX_pydx(e)') ###
# current_medium.remove('EX_pydxn(e)')
# current_medium.remove('EX_rbflvrd(e)')
# current_medium.remove('EX_ribflv(e)')
current_medium.remove('EX_seln(e)')
# current_medium.remove('EX_ser_L(e)')
# current_medium.remove('EX_so4(e)') ###
current_medium.remove('EX_spmd(e)')
# current_medium.remove('EX_strch1(e)')
current_medium.remove('EX_succ(e)')
current_medium.remove('EX_sulfac(e)')
# current_medium.remove('EX_taur(e)')
# current_medium.remove('EX_thm(e)')
# current_medium.remove('EX_thr_L(e)')
current_medium.remove('EX_thymd(e)')
# current_medium.remove('EX_trp_L(e)')
# current_medium.remove('EX_tyr_L(e)')
current_medium.remove('EX_ura(e)')
current_medium.remove('EX_urate(e)')
current_medium.remove('EX_urea(e)')
# current_medium.remove('EX_val_L(e)')
current_medium.remove('EX_xan(e)')

In [8]:
# Get molar masses from pubchem
# This function takes some time to run, but it gets all of the molar masses for the metabolites in the media
# The result is a dataframe that we'll use to manipulate concentrations and eventually calculate flux lower bounds, but for now we
# need to deal with all of the molar masses that couldn't be found
weights = get_molecular_weights(root_model, current_medium)
weights.head()


,query,BiGG_ID,hit,mol_weight
0,L-cysteate,Lcyst(e),23619007,168.15
1,L-Cystine,Lcystin(e),67678,240.3
2,Acetate,ac(e),175,59.04
3,N-acetyl-L-aspartate,acasp(e),65065,175.14
4,N-Acetyl-D-galactosamine,acgal(e),35717,221.21


In [9]:
# Still missing molar masses:
weights[weights['mol_weight'] == 'NA']

,query,BiGG_ID,hit,mol_weight
27,"Capsular polysaccharide (F. prausnitzii, putat...",cps_fp(e),NA,NA
44,exchange reaction for glycylglyci,glygly(e),NA,NA
47,exchange reaction for glycylphenylalai,glyphe(e),NA,NA
56,exchange reaction for leucylglyci,leugly(e),NA,NA
69,exchange reaction for pectins,pect(e),NA,NA
81,"starch, structure 1 (1,6-{7[1,4-Glc], 4[1,4-Gl...",strch1(e),NA,NA


In [10]:
### MANUALLY fill these in
# I just did this by adding together the molar mass of each of the metabolites in the exchange reaction
weights.loc[weights['BiGG_ID'] == 'glygly(e)', 'mol_weight'] = 75.07*2
weights.loc[weights['BiGG_ID'] == 'glyphe(e)', 'mol_weight'] = 75.07+165.19
weights.loc[weights['BiGG_ID'] == 'leugly(e)', 'mol_weight'] = 75.07+131.17
weights.loc[weights['BiGG_ID'] == 'pect(e)', 'mol_weight'] = 194.14
weights.loc[weights['BiGG_ID'] == 'strch1(e)', 'mol_weight'] = 359.33
weights.loc[weights['BiGG_ID'] == 'cps_fp(e)', 'mol_weight'] = 1000

# There should be nothing left; everything now has a molar mass
weights[weights['mol_weight'] == 'NA']

,query,BiGG_ID,hit,mol_weight


In [11]:
# Here, we set up the dataframes for some of the main components of the media: peptones and yeast extracts
# For traces, each entry is named EXACTLY after a BiGG metabolite, so the concentration can be added to this metabolite exclusively
# 
# For amino acids (AAs), each entry is named after a substring corresponding to an AA(asp, arg, leu, etc.). 
# This is because we have no way of telling how much of each form of AA and how to differentiate their proportion in the media 
# (ie. how much L-cysteate do we have compared to L-cysteine? We really don't know)
# There is a function that does this for us, and is described in more detail in the cells below
#
# These tables are based on the oxoid data:
# http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN&minfo=Y
# All values are in ppm
traces_list = [{'BiGG_ID':'ca2(e)', "conc_ppm_Bact_Peptone_L37": 635, "conc_ppm_Proteose_Peptone_L85": 200, "conc_ppm_soya_peptone_L44": 225, "conc_ppm_yeast_extract": 155},
{'BiGG_ID':'mg2(e)', "conc_ppm_Bact_Peptone_L37": 265, "conc_ppm_Proteose_Peptone_L85": 340, "conc_ppm_soya_peptone_L44": 1530, "conc_ppm_yeast_extract": 205},
{'BiGG_ID':'fe3(e)', "conc_ppm_Bact_Peptone_L37": 22, "conc_ppm_Proteose_Peptone_L85": 42, "conc_ppm_soya_peptone_L44": 90, "conc_ppm_yeast_extract": 52},
{'BiGG_ID':'cu2(e)', "conc_ppm_Bact_Peptone_L37": 1, "conc_ppm_Proteose_Peptone_L85": 10, "conc_ppm_soya_peptone_L44": 2, "conc_ppm_yeast_extract": 2},
{'BiGG_ID':'pb(e)', "conc_ppm_Bact_Peptone_L37": 0.4, "conc_ppm_Proteose_Peptone_L85": 0.1, "conc_ppm_soya_peptone_L44": 0.3, "conc_ppm_yeast_extract": 0.7},
{'BiGG_ID':'mn2(e)', "conc_ppm_Bact_Peptone_L37": 3.4, "conc_ppm_Proteose_Peptone_L85": 0.5, "conc_ppm_soya_peptone_L44": 1.0, "conc_ppm_yeast_extract": 1.3},
{'BiGG_ID':'zn2(e)', "conc_ppm_Bact_Peptone_L37": 9.2, "conc_ppm_Proteose_Peptone_L85": 16, "conc_ppm_soya_peptone_L44": 12, "conc_ppm_yeast_extract": 94},
{'BiGG_ID':'cobalt2(e)', "conc_ppm_Bact_Peptone_L37": 0.1, "conc_ppm_Proteose_Peptone_L85": 0.2, "conc_ppm_soya_peptone_L44": 0.2, "conc_ppm_yeast_extract": 3.1},
{'BiGG_ID':'cl(e)', "conc_ppm_Bact_Peptone_L37": 10000, "conc_ppm_Proteose_Peptone_L85": 80000, "conc_ppm_soya_peptone_L44": 4000, "conc_ppm_yeast_extract": 3000},
{'BiGG_ID':'k(e)', "conc_ppm_Bact_Peptone_L37": 36000, "conc_ppm_Proteose_Peptone_L85": 14000, "conc_ppm_soya_peptone_L44": 33000, "conc_ppm_yeast_extract": 73000},
{'BiGG_ID':'na1(e)', "conc_ppm_Bact_Peptone_L37": 10000, "conc_ppm_Proteose_Peptone_L85": 80000, "conc_ppm_soya_peptone_L44": 4000, "conc_ppm_yeast_extract": 3000}]


aa_list = [{'BiGG_ID':'ala', "conc_ppm_Bact_Peptone_L37": 39200 , "conc_ppm_Proteose_Peptone_L85": 38100, "conc_ppm_soya_peptone_L44": 28700, "conc_ppm_yeast_extract": 9100},
{'BiGG_ID':'arg', "conc_ppm_Bact_Peptone_L37": 49900, "conc_ppm_Proteose_Peptone_L85": 58000, "conc_ppm_soya_peptone_L44": 46400, "conc_ppm_yeast_extract": 33100},
{'BiGG_ID':'asp', "conc_ppm_Bact_Peptone_L37": 60600, "conc_ppm_Proteose_Peptone_L85": 58500, "conc_ppm_soya_peptone_L44": 70600, "conc_ppm_yeast_extract": 70700}, 
{'BiGG_ID':'cys', "conc_ppm_Bact_Peptone_L37": 16600, "conc_ppm_Proteose_Peptone_L85": 1500, "conc_ppm_soya_peptone_L44": 5300, "conc_ppm_yeast_extract": 7600},
{'BiGG_ID':'glu', "conc_ppm_Bact_Peptone_L37": 99300, "conc_ppm_Proteose_Peptone_L85": 137800, "conc_ppm_soya_peptone_L44": 147100, "conc_ppm_yeast_extract": 134900},
{'BiGG_ID':'gly', "conc_ppm_Bact_Peptone_L37": 77100 , "conc_ppm_Proteose_Peptone_L85": 44500, "conc_ppm_soya_peptone_L44": 28300, "conc_ppm_yeast_extract": 59500},
{'BiGG_ID':'ile', "conc_ppm_Bact_Peptone_L37": 38100, "conc_ppm_Proteose_Peptone_L85": 45800, "conc_ppm_soya_peptone_L44": 25100, "conc_ppm_yeast_extract": 48100},
{'BiGG_ID':'leu', "conc_ppm_Bact_Peptone_L37": 37900, "conc_ppm_Proteose_Peptone_L85": 60100, "conc_ppm_soya_peptone_L44": 43100, "conc_ppm_yeast_extract": 60400},
{'BiGG_ID':'lys', "conc_ppm_Bact_Peptone_L37": 43800, "conc_ppm_Proteose_Peptone_L85": 46100, "conc_ppm_soya_peptone_L44": 37700, "conc_ppm_yeast_extract": 54000},
{'BiGG_ID':'met', "conc_ppm_Bact_Peptone_L37": 15800, "conc_ppm_Proteose_Peptone_L85": 10800, "conc_ppm_soya_peptone_L44": 6200, "conc_ppm_yeast_extract": 8000},
{'BiGG_ID':'phe', "conc_ppm_Bact_Peptone_L37": 26000, "conc_ppm_Proteose_Peptone_L85": 46600, "conc_ppm_soya_peptone_L44": 3800, "conc_ppm_yeast_extract": 37800},
{'BiGG_ID':'pro', "conc_ppm_Bact_Peptone_L37": 58300, "conc_ppm_Proteose_Peptone_L85": 59900, "conc_ppm_soya_peptone_L44": 34000, "conc_ppm_yeast_extract": 8800},
{'BiGG_ID':'ser', "conc_ppm_Bact_Peptone_L37": 28100, "conc_ppm_Proteose_Peptone_L85": 21800, "conc_ppm_soya_peptone_L44": 6700, "conc_ppm_yeast_extract": 34200},
{'BiGG_ID':'thr', "conc_ppm_Bact_Peptone_L37": 12500, "conc_ppm_Proteose_Peptone_L85": 27500, "conc_ppm_soya_peptone_L44": 16800, "conc_ppm_yeast_extract": 27300},
{'BiGG_ID':'trp', "conc_ppm_Bact_Peptone_L37": 6600, "conc_ppm_Proteose_Peptone_L85": 7500, "conc_ppm_soya_peptone_L44": 6400, "conc_ppm_yeast_extract": 8500},
{'BiGG_ID':'tyr', "conc_ppm_Bact_Peptone_L37": 3900, "conc_ppm_Proteose_Peptone_L85": 17700, "conc_ppm_soya_peptone_L44": 20900, "conc_ppm_yeast_extract": 49500},
{'BiGG_ID':'val', "conc_ppm_Bact_Peptone_L37": 33300, "conc_ppm_Proteose_Peptone_L85": 41100, "conc_ppm_soya_peptone_L44": 36500, "conc_ppm_yeast_extract": 10000}]

#https://www.sigmaaldrich.com/catalog/product/sial/07533?lang=de&region=DE 
#Vitamins mg/100 g product conversion factor to ppm 1/10000
#Vitamin B6 consists of 3 components see also wikipedia and vitamin B3 2 components https://en.wikipedia.org/wiki/B_vitamins 

vitamins_list=[{'BiGG_ID':'thm(e)', "conc_ppm_Bact_Peptone_L37": 36/10000, "conc_ppm_Proteose_Peptone_L85": 36/10000, "conc_ppm_soya_peptone_L44": 36/10000, "conc_ppm_yeast_extract": 36/10000},
{'BiGG_ID':'ribflv(e)', "conc_ppm_Bact_Peptone_L37": 13/2/10000, "conc_ppm_Proteose_Peptone_L85": 13/2/10000, "conc_ppm_soya_peptone_L44": 13/2/10000, "conc_ppm_yeast_extract": 13/2/10000}, 
{'BiGG_ID':'rbflvrd(e)', "conc_ppm_Bact_Peptone_L37":13/2/10000, "conc_ppm_Proteose_Peptone_L85": 13/2/10000, "conc_ppm_soya_peptone_L44":13/2/10000, "conc_ppm_yeast_extract": 13/2/10000},
{'BiGG_ID':'nac(e)', "conc_ppm_Bact_Peptone_L37": 650/2/10000, "conc_ppm_Proteose_Peptone_L85": 650/2/10000, "conc_ppm_soya_peptone_L44": 650/2/10000, "conc_ppm_yeast_extract": 650/2/10000},
{'BiGG_ID':'ncam(e)', "conc_ppm_Bact_Peptone_L37": 650/2/10000 , "conc_ppm_Proteose_Peptone_L85": 650/2/10000, "conc_ppm_soya_peptone_L44": 650/2/10000, "conc_ppm_yeast_extract":650/2/10000},
{'BiGG_ID':'pydam(e)', "conc_ppm_Bact_Peptone_L37": 3.9/3/10000, "conc_ppm_Proteose_Peptone_L85": 3.9/3/10000, "conc_ppm_soya_peptone_L44": 3.9/3/10000, "conc_ppm_yeast_extract": 3.9/3/10000},
{'BiGG_ID':'pydx(e)', "conc_ppm_Bact_Peptone_L37": 3.9/3/10000+7/10000, "conc_ppm_Proteose_Peptone_L85": 3.9/3/10000+7/10000, "conc_ppm_soya_peptone_L44": 3.9/3/10000+7/10000, "conc_ppm_yeast_extract": 3.9/3/10000+7/10000},
{'BiGG_ID':'pydxn(e)', "conc_ppm_Bact_Peptone_L37": 3.9/3/10000, "conc_ppm_Proteose_Peptone_L85": 3.9/3/10000, "conc_ppm_soya_peptone_L44": 3.9/3/10000, "conc_ppm_yeast_extract": 3.9/3/10000},
{'BiGG_ID':'fol(e)', "conc_ppm_Bact_Peptone_L37": 2.8/10000, "conc_ppm_Proteose_Peptone_L85": 2.8/10000, "conc_ppm_soya_peptone_L44": 2.8/10000, "conc_ppm_yeast_extract": 2.8/10000},
{'BiGG_ID':'pnto_R(e)', "conc_ppm_Bact_Peptone_L37": 11.2/10000 , "conc_ppm_Proteose_Peptone_L85": 11.2/10000, "conc_ppm_soya_peptone_L44": 11.2/10000, "conc_ppm_yeast_extract": 11.2/10000},
{'BiGG_ID':'btn(e)', "conc_ppm_Bact_Peptone_L37": 2/10000, "conc_ppm_Proteose_Peptone_L85": 2/10000, "conc_ppm_soya_peptone_L44": 2/10000, "conc_ppm_yeast_extract": 2/10000}]

medium_trace = pd.DataFrame(traces_list+vitamins_list)
medium_aa = pd.DataFrame(aa_list)

In [12]:
# This cell does a dataframe merge (essentially a SQL left join)
# It combines the columns from the dataframe that contains each of the metabolites in the medium defined earlier and
# the columns for each of the metabolite concentrations in each of the oxoid media ingredients
# Note: this only deals with the "traces" df; the amino acids will be dealt with in the next operation
weights_and_traces = weights.merge(medium_trace, how="left", on="BiGG_ID")
weights_and_traces[weights_and_traces['conc_ppm_Bact_Peptone_L37'] > 0]

,query,BiGG_ID,hit,mol_weight,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract
22,Biotin,btn(e),171548,244.31,0.00020,0.00020,0.00020,0.00020
23,Calcium,ca2(e),5460341,40.08,635.00000,200.00000,225.00000,155.00000
25,Chloride,cl(e),312,35.45,10000.00000,80000.00000,4000.00000,3000.00000
26,Co2+,cobalt2(e),104729,58.9332,0.10000,0.20000,0.20000,3.10000
28,Cu2+,cu2(e),27099,63.55,1.00000,10.00000,2.00000,2.00000
30,Fe3+,fe3(e),29936,55.84,22.00000,42.00000,90.00000,52.00000
31,Folate,fol(e),135398658,441.4,0.00028,0.00028,0.00028,0.00028
54,K+,k(e),813,39.098,36000.00000,14000.00000,33000.00000,73000.00000
64,Mg,mg2(e),5462224,24.305,265.00000,340.00000,1530.00000,205.00000
65,Sodium,na1(e),5360545,22.9898,10000.00000,80000.00000,4000.00000,3000.00000


In [13]:
# Returning to the problem outlined above when we were declaring the dictionaries full of ppm data above:
# How much L-cysteate do we have compared to L-cysteine?
# The solution to this is to take every metabolite containing this substring and assign it to an equal proportion of the given concentration in the media.
# 
# (ie. If the technical data says that there is 16600 ppm of cysteine in bacterial peptone, and we assume that our media contains L-cysteine and L-cysteate that # contain the "cys" substring, they will each get an equal portion of that 16600ppm: 8300ppm each. 
# This scales independent of the number of "cys" variants we assume to be in the media.
# So if there are 4 metabolites containing the "cys" substring, they will each be given 4150ppm from the 16600ppm, and so on.)
#
# The function "split_concentration_proportions()" does this for us. It's in the file "functions.py" and has some insight 
# into what the input and return values should be.
weights_full = split_concentration_proportions(weights_and_traces, medium_aa)
weights_full.head()

,query,BiGG_ID,hit,mol_weight,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract
0,L-cysteate,Lcyst(e),23619007,168.15,5533.333333,500.0,1766.666667,2533.333333
1,L-Cystine,Lcystin(e),67678,240.30,5533.333333,500.0,1766.666667,2533.333333
2,Acetate,ac(e),175,59.04,0.000000,0.0,0.000000,0.000000
3,N-acetyl-L-aspartate,acasp(e),65065,175.14,15150.000000,14625.0,17650.000000,17675.000000
4,N-Acetyl-D-galactosamine,acgal(e),35717,221.21,0.000000,0.0,0.000000,0.000000


It's easier to have this table here for now, for the cell below

| Component                            | Concentration (g/L) |
|--------------------------------------|---------------------|
| Bacteriological Peptome              | 6.5                 |
| Soy Peptome                          | 5.0                 |
| Yeast Extract                        | 3.0                 |
| Tryptone                             | 2.5                 |
| NaCL                                 | 1.5                 |
| K<sub>2</sub>PO<sub>4</sub>          | 1.0                 |
| KH<sub>2</sub>PO<sub>4</sub>         | 1.0                 |
| Na<sub>2</sub>SO<sub>4</sub>         | 2.0                 |
| Mg<sub>2</sub>SO<sub>4</sub>         | 1.0                 |
| CaCl<sub>2</sub>*2H<sub>2</sub>O     | 0.1                 |
| NH<sub>4</sub>Cl                     | 1.0                 |
| Cysteine-HCL                         | 0.4                 |
| NaHCO<sub>3</sub>                    | 0.2                 |
| MnSO<sub>4</sub>*H<sub>2</sub>O      | 0.05                |
| FeSOMnSO<sub>4</sub>*7H<sub>2</sub>O | 0.005               |
| ZnSO<sub>4</sub>*7H<sub>2</sub>O     | 0.005               |
| Hemin                                | 0.005               |
| Menadione                            | 0.005               |
| Resazurin                            | 0.001               |


In [14]:
# Assign proportions based on mmcb medium concentrations:
# This is pretty subjective and makes a lot of assumptions about dissociation reactions of these medium components
# (ie. what happens to Cysteine-HCL in the medium? The model doesn't use Cysteine-HCL in any reaction, but it's added intentionally, so it must be a source
# of some variant of cysteine)
peptone_columns = {"conc_ppm_Bact_Peptone_L37": 6500, "conc_ppm_Proteose_Peptone_L85": 0, "conc_ppm_soya_peptone_L44": 5000, "conc_ppm_yeast_extract": 3000}
update = {'trp': 2500, '^ac': 6800, 'cys': 400}
supplements = {'na1(e)': 750, 'cl(e)':750, 'pi(e)': 660, 'k(e)': 660, 'h(e)': 660, 'so4(e)': 660, 'mg2(e)': 660, 'ca2(e)': 33, 'nh4(e)': 1000, 'fe3(e)': 2750, 'cobalt2(e)': 190, 'cu2(e)': 3, 'h2o(e)': 1000000, 'for(e)': 3400}

# Scale by peptone columns ~~~
# peptone_columns: each of the components from the Oxoid table. For example, there are 6.5g/L of bacterial peptone in the media, which we've input as 6500 (ppm). 
# We'll use this to scale each of the corresponding columns to this g/L proportion:

total_concentration = sum([sum(i.values()) for i in [peptone_columns, update, supplements]])
for i in peptone_columns.keys():
    scale_value = peptone_columns[i]/total_concentration
    weights_full[i] = weights_full[i]*scale_value

# Update ambiguous supplement metabolites ~~~
# For the rest of the medium components, their concentrations will be added to a new column in the dataframe called "supplements"
# Using the "update" dictionary, we'll search for the key (ie. 'trp', 'cys') that is present in our defined media
# Collecting all of the metabolites corresponding to these substrings, each individual metabolite will be given an extra amount in the "supplements" column
#
# (Similar to before: if L-cysteine and L-cysteate are the two metabolites corresponding to the "cys" substring and there's a 400ppm source of "cys", each 
# metabolite will be assigned 200 (ppm) in the "supplements" column)

# Note: '^ac' is used because there was a 6.8g/L source of "acetate". The hat is there to signify that we're looking for metabolites that start with 'ac' as to 
# not accidentally detect 'nac(e)'

weights_full['supplements'] = 0
for i in update.keys():
    aa_list = weights_full.loc[weights_full["BiGG_ID"].str.contains(i), "BiGG_ID"]
    aa_count = len(aa_list)
    proportion = update[i]/aa_count
    for aa in aa_list:
        weights_full.loc[weights_full["BiGG_ID"] == aa, "supplements"] = proportion

# Add supplements ~~~
# Finally, directly add the concentrations of metabolites to the "supplements" column 

for i in supplements.keys():
    weights_full.loc[weights_full["BiGG_ID"] == i, "supplements"] = supplements[i]

weights_full.head()

,query,BiGG_ID,hit,mol_weight,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract,supplements
0,L-cysteate,Lcyst(e),23619007,168.15,34.704264,0.0,8.523290,7.333246,133.333333
1,L-Cystine,Lcystin(e),67678,240.30,34.704264,0.0,8.523290,7.333246,133.333333
2,Acetate,ac(e),175,59.04,0.000000,0.0,0.000000,0.000000,1133.333333
3,N-acetyl-L-aspartate,acasp(e),65065,175.14,95.018603,0.0,85.152493,51.163863,1133.333333
4,N-Acetyl-D-galactosamine,acgal(e),35717,221.21,0.000000,0.0,0.000000,0.000000,1133.333333


In [15]:
# There are still some 0's left in the medium, meaning that there are still some metabolites used in the reaction that don't have a concentration defined in
# the medium from the literature (that I've been able to figure out) so this is where I've taken the mean value for each of the concentration columns
# and used that to fill in any 0's in that column
# This is a huge, baseless assumption; I think we might be better off manually determining which metabolites that have a concentration are similar to metabolites # that don't have a concentration.

# Note: I don't think this would be a huge problem normally, but there are some metabolites here that are necessary for growth 
# according to the "mandatory" metabolites from earlier
print(len(weights_full[weights_full.iloc[:,4:10].sum(axis=1) == 0]))
display(weights_full[weights_full.iloc[:,4:10].sum(axis=1) == 0])

undef_list = weights_full.loc[weights_full.iloc[:,4:10].sum(axis=1) == 0, "BiGG_ID"]
fill_value = {}
# Mean averaging for each of the columns:
for media in weights_full.columns[4:]:
    fill_value[media] = weights_full[media].mean()

for met in undef_list:
    for media in fill_value.keys():
        weights_full.loc[weights_full['BiGG_ID'] == met, media] = fill_value[media]

12


,query,BiGG_ID,hit,mol_weight,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract,supplements
8,Adenine,ade(e),190,135.13,0.0,0.0,0.0,0.0,0.0
20,L-Asparagine,asn_L(e),6267,132.12,0.0,0.0,0.0,0.0,0.0
27,"Capsular polysaccharide (F. prausnitzii, putat...",cps_fp(e),NA,1000.00,0.0,0.0,0.0,0.0,0.0
33,D-Glucosamine,gam(e),439213,179.17,0.0,0.0,0.0,0.0,0.0
34,D-Glucose,glc(e),5793,180.16,0.0,0.0,0.0,0.0,0.0
35,D-Glucuronate,glcur(e),94715,194.14,0.0,0.0,0.0,0.0,0.0
36,L-Glutamine,gln_L(e),5961,146.14,0.0,0.0,0.0,0.0,0.0
50,Guanine,gua(e),135398634,151.13,0.0,0.0,0.0,0.0,0.0
69,exchange reaction for pectins,pect(e),NA,194.14,0.0,0.0,0.0,0.0,0.0
81,"starch, structure 1 (1,6-{7[1,4-Glc], 4[1,4-Gl...",strch1(e),NA,359.33,0.0,0.0,0.0,0.0,0.0


In [16]:
# Making calculations to eventually get to flux:
weights_full['g/L'] = weights_full.iloc[:,4:9].sum(axis=1)/1000 # ppm/1000 = g/L (roughly)
weights_full['mmol/mL'] = weights_full['g/L']/weights_full['mol_weight'] # (g/L)/(molar mass) = mol/L == mmol/mL
weights_full.head()

,query,BiGG_ID,hit,mol_weight,conc_ppm_Bact_Peptone_L37,conc_ppm_Proteose_Peptone_L85,conc_ppm_soya_peptone_L44,conc_ppm_yeast_extract,supplements,g/L,mmol/mL
0,L-cysteate,Lcyst(e),23619007,168.15,34.704264,0.0,8.523290,7.333246,133.333333,0.183894,0.001094
1,L-Cystine,Lcystin(e),67678,240.30,34.704264,0.0,8.523290,7.333246,133.333333,0.183894,0.000765
2,Acetate,ac(e),175,59.04,49.605690,0.0,32.874253,24.098164,1133.333333,1.239911,0.021001
3,N-acetyl-L-aspartate,acasp(e),65065,175.14,95.018603,0.0,85.152493,51.163863,1133.333333,1.364668,0.007792
4,N-Acetyl-D-galactosamine,acgal(e),35717,221.21,49.605690,0.0,32.874253,24.098164,1133.333333,1.239911,0.005605


In [37]:
# Cells per mL of medium: cells_per_ml = 1640992
# file: FP_14.xlsx, sheet: Determination of Cell Count, excel cell: P4
# updated: FP_14.xlsx, Determination of Cell Count, excell cell: P11
cells_per_ml = 1001930111

time = 80
# gDw: based on EColi gDw
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC106103/
# 1172 fg EColi ~= 1172E-15 g
gdw_per_cell = 1172E-15

# Calculating flux per metabolite in the medium based on the reference equation (flux = mmol/gDw)
weights_full['flux'] = weights_full['mmol/mL']/(cells_per_ml * gdw_per_cell * time)


In [38]:
# FINALLY
# Use the "flux" column from the "weights_full" df to set the lower bound for each of the metabolites in the model 
print("mMCB")
for met in root_model.exchanges:
    if '_o2(e)' in met.id:
        root_model.reactions.get_by_id(met.id).lower_bound = 0
    else:
        root_model.reactions.get_by_id(met.id).lower_bound = -0.001

for id in ["EX_"+metab for metab in list(weights_full['BiGG_ID'])]:
    root_model.reactions.get_by_id(id).upper_bound = 1000
    root_model.reactions.get_by_id(id).lower_bound = weights_full.loc[weights_full["BiGG_ID"] == id.lstrip("EX_"), "flux"].values[0]*-1
root_model.objective = {root_model.reactions.get_by_id('Biomass_FP'): 1}
root_model.optimize()

mMCB


,fluxes,reduced_costs
26DAPLLATi,0.037884,-1.577722e-30
3HAD100,0.092006,0.000000e+00
3HAD120,0.054843,-1.262177e-29
3HAD121,0.037163,-1.262177e-29
3HAD140,0.052317,0.000000e+00
...,...,...
XYLt2,0.000000,-3.338268e-16
YUMPS,0.000000,0.000000e+00
r0502,0.000000,0.000000e+00
r0839,0.001000,-5.084281e-16


In [39]:
root_model.slim_optimize()

2.1683617789587515e-08

In [18]:
# Model outputs
root_model.summary().to_frame().to_csv('mmcb_model_summary.csv')
# root_model.summary().to_frame()

In [19]:
# Post-mortem
# Each metabolite in the model, its flux, and how much growth decreases when it's removed from the medium (one of the first steps)
# Sorted by growth (descending), we can take a look at the "most important" metabolites and what their fluxes were in the medium
# weights_full[['query','BiGG_ID','flux']].merge(growth_check_df, how="left", left_on="BiGG_ID", right_on="metab").sort_values(by="BiGG_ID")

In [20]:
# Intended inputs
weights_full[['BiGG_ID','flux']].to_csv('mmcb.csv')

In [21]:
root_model.reactions.get_by_id('EX_for(e)')

Reaction identifier,EX_for(e)
Name,Formate exchange
Memory address,0x07f8b4e86b550
Stoichiometry,for[e] <=> Formate <=>
GPR,
Lower bound,-0.829185960117312
Upper bound,1000


In [24]:
fba=root_model.optimize()
for rxn in root_model.exchanges:
    #print(rxn.id,rxn.lower_bound,fba.fluxes[rxn.id])
    if fba.fluxes[rxn.id]>= rxn.lower_bound and fba.fluxes[rxn.id]<=rxn.lower_bound-0.01*rxn.lower_bound and rxn.lower_bound<0:
        print(rxn.id,rxn.lower_bound, fba.fluxes[rxn.id] )

EX_ade(e) -0.9128808238059144 -0.9128808238059144
EX_btn(e) -1.2192264610729274e-10 -1.2192264610729274e-10
EX_cellb(e) -0.001 -0.001
EX_fru(e) -0.001 -0.001
EX_fum(e) -0.001 -0.001
EX_glc(e) -0.68471128841526 -0.68471128841526
EX_ins(e) -0.001 -0.001
EX_inulin(e) -0.001 -0.001
EX_mal_L(e) -0.001 -0.001
EX_na1(e) -0.3892707342126576 -0.3892707342126576
EX_ser_L(e) -0.031154187322075883 -0.031154187322075883


In [33]:
# Manually extracted outputs
csv_list = []
for rxn in root_model.exchanges:
    csv_dict = {}
    csv_dict["metabolite"] = rxn.id
    csv_dict["lower_bound"] = rxn.lower_bound
    csv_dict["upper_bound"] = rxn.upper_bound
    csv_dict["flux"] = rxn.flux
    csv_list.append(csv_dict)

pd.DataFrame(csv_list).to_csv("mmcb_file_final.csv")

In [ ]:
mandatory.to_csv('mandatory_mmcb.csv', index=False)